In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P10', 'P11', 'P12', 'P13', 'P14']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    #clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P2.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    !rm autoclass.r-params
    !cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/Hp3.csv', index = True, header=True)

2020-04-27 15:22:12 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P10.tsv' as 'real location' with error 0.01
2020-04-27 15:22:12 INFO     Detected encoding: ascii
2020-04-27 15:22:12 INFO     Found 3200 rows and 2 columns
2020-04-27 15:22:12 DEBUG    Checking column names
2020-04-27 15:22:12 DEBUG    Index name 'index'
2020-04-27 15:22:12 DEBUG    Column name 'P10'
2020-04-27 15:22:12 INFO     Checking data format
2020-04-27 15:22:12 INFO     Column 'P10'
2020-04-27 15:22:12 INFO     count    3200.000000
2020-04-27 15:22:12 INFO     mean       -0.012175
2020-04-27 15:22:12 INFO     std         2.500738
2020-04-27 15:22:12 INFO     min       -42.287518
2020-04-27 15:22:12 INFO     50%         0.174727
2020-04-27 15:22:12 INFO     max         8.556823
2020-04-27 15:22:12 INFO     ---
2020-04-27 15:22:12 INFO     Final dataframe has 3200 lines and 2 columns
2020-04-27 15:22:12 INFO     Searching for missing values
2020-04-27 15:22:12 IN

Time: 142 sec.

2020-04-27 15:24:35 INFO     Extracting autoclass results
2020-04-27 15:24:35 INFO     Found 3200 cases classified in 10 classes
2020-04-27 15:24:39 INFO     Aggregating input data
2020-04-27 15:24:39 INFO     Writing classes + probabilities .tsv file
2020-04-27 15:24:39 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P11.tsv' as 'real location' with error 0.01
2020-04-27 15:24:39 INFO     Detected encoding: ascii
2020-04-27 15:24:39 INFO     Found 3200 rows and 2 columns
2020-04-27 15:24:39 DEBUG    Checking column names
2020-04-27 15:24:39 DEBUG    Index name 'index'
2020-04-27 15:24:39 DEBUG    Column name 'P11'
2020-04-27 15:24:39 INFO     Checking data format
2020-04-27 15:24:39 INFO     Column 'P11'
2020-04-27 15:24:39 INFO     count    3200.000000
2020-04-27 15:24:39 INFO     mean       -0.187907
2020-04-27 15:24:39 INFO     std         1.310756
2020-04-27 15:24:39 INFO     min       -11.829763
2020-04-27 15:24:39 INFO     50%   

[0.162839248434238, 0.1592920353982301, 0.054455445544554455, 0.06891891891891892, 0.07588075880758807, 0.22160664819944598, 0.07772020725388601, 0.17777777777777778, 0.13793103448275862, 0.4]
['DNS', 'DROPBOX', 'WINDOWS_UPDATE', 'AMAZON', 'SPOTIFY', 'IP_ICMP', 'SPOTIFY', 'DNS', 'HTTP', 'APPLE_ICLOUD']
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11]
0.1536422074817398


2020-04-27 15:24:40 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:24:40 INFO     Writing run file
2020-04-27 15:24:40 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:24:40 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 15:24:40 INFO     Running clustering...


...

[0.05012531328320802, 0.0743362831858407, 0.11422845691382766, 0.3723404255319149, 0.2553191489361702]
['IP_ICMP', 'OFFICE_365', 'CONTENT_FLASH', 'DROPBOX', 'FACEBOOK']
[1, 2, 3, 4, 5]
0.1732699255701923


2020-04-27 15:34:54 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:34:54 INFO     Writing run file
2020-04-27 15:34:54 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 15:34:54 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 15:34:54 INFO     Running clustering...


Time: 194 sec.

2020-04-27 15:38:08 INFO     Extracting autoclass results
2020-04-27 15:38:08 INFO     Found 3200 cases classified in 8 classes
2020-04-27 15:38:11 INFO     Aggregating input data
2020-04-27 15:38:11 INFO     Writing classes + probabilities .tsv file


[0.06058091286307054, 0.06666666666666667, 0.11885245901639344, 0.06921241050119331, 0.07419354838709677, 0.0859375, 0.7122302158273381, 0.17424242424242425]
['DNS', 'MS_ONE_DRIVE', 'DROPBOX', 'CLOUDFLARE', 'HTTP', 'EBAY', 'IP_ICMP', 'APPLE']
[1, 2, 3, 4, 5, 6, 7, 8]
0.17023951718802288
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5 non-null      object 
 1   1       5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes
